In [ ]:
/*

在這個Imperative的編寫style Demo中，
我們會以scalaj-http抓取104關於java的數據
並透過play-json抓出各工作的地點，並統計出各地區對於java工程師的需求量
再透過vegas，做資料可視化

*/

In [1]:
// 載入套件
classpath.add(

  "org.scalaj" %% "scalaj-http" % "2.3.0",
  "com.typesafe.play" %% "play-json" % "2.6.2",
  
  "org.vegas-viz" %% "vegas" % "0.3.9",
  
  "org.http4s" %% "http4s-dsl" % "0.15.16a",
  "org.http4s" %% "http4s-blaze-server" % "0.15.16a",
  "org.http4s" %% "http4s-blaze-client" % "0.15.16a"
)

Adding 50 artifact(s)


In [2]:
// 先對104的service發一個小request，得知java的工作資料總筆數，而後用迴圈爬完各個分頁
import scalaj.http._
val response: HttpResponse[String] = Http("http://www.104.com.tw/i/apis/jobsearch.cfm")
  .param("kws","java")
  .param("kwop","3")
  .param("role","0")
  .param("fmt","8")
  .param("page","1")
  .param("pgsz","50")
  .asString

// 轉換成Json作處理
import play.api.libs.json._

val resultJson = Json.parse(response.body)
 
// 找出總筆數
val jobSize = resultJson.\("RECORDCOUNT").as[String].toInt

// 以一頁兩千筆的方式計算總共要訪問幾次
val freq = (jobSize / 100) +1


import scalaj.http._
response: scalaj.http.HttpResponse[String] = HttpResponse(
  """
{"RECORDCOUNT":"4658","PAGECOUNT":"50","PAGE":"1","TOTALPAGE":"94","data":[{"ADDRESS":"林森南路4之2號7樓","ADDR_INDZONE":"","ADDR_NO_DESCRIPT":"台北市中正區","APPEAR_DATE":"20170819","APPEAR_TIME":"20170819180509","C":"383a43293c363e683c423b1d1d1d1d5f2443a363189j56","CERT_ALL_DESCRIPT":"","DESCRIPTION":"1.建置與開發網站前後台\n2.資料庫建置與規劃\n3.程式開發與維護\n4.熟悉.net(C#、VB) C#尤佳，及MS SQL\n5.熟悉Ajax、Java Script、CSS等相關網頁開發技術\n6.熟MVC開發架構佳\n7.具備三層式架構，N-Tier及OOP經驗，若有Team Foundation 經驗佳","DIS_LEVEL":"0","DIS_LEVEL2":"0","DIS_LEVEL3":"0","DIS_ROLE":"0","DIS_ROLE2":"0","DIS_ROLE3":"0","DRIVER":"0","HANDICOMPENDIUM":"0","INDCAT":"資訊軟體服務","J":"5d49426c335b3e55303139622b2d361c93f39416b3f453e2020202064373a2524290j00","JOB":"Asp.Net ,C# 程式設計師","JOBCAT_DESCRIPT":"軟體設計工程師@Internet程式設計師@MIS程式設計師","JOBSKILL_ALL_DESC":"","JOB_ADDRESS":"林森南路4之2號7樓","JOB_ADDR_NO_DESCRIPT":"台北市中正區","LANGUAGE1":"001111","LANGUAGE2":"001111","LANGUAGE3":"001111","LAT":"25.0

In [3]:
// 將所爬下來的資料，放入Array中
// 傳統迴圈的方式
import scala.collection.mutable.ArrayBuffer

var traditionArray = ArrayBuffer[String]()

for (i <- 1 to freq){
  val t = Http("http://www.104.com.tw/i/apis/jobsearch.cfm")
  .param("kws","java")
  .param("kwop","3")
  .param("role","0")
  .param("fmt","8")
  .param("page",s"$i")
  .param("pgsz","50")
  .asString
  traditionArray += t.body
}

import scala.collection.mutable.ArrayBuffer
traditionArray: collection.mutable.ArrayBuffer[String] = ArrayBuffer(
  """
{"RECORDCOUNT":"4658","PAGECOUNT":"50","PAGE":"1","TOTALPAGE":"94","data":[{"ADDRESS":"林森南路4之2號7樓","ADDR_INDZONE":"","ADDR_NO_DESCRIPT":"台北市中正區","APPEAR_DATE":"20170819","APPEAR_TIME":"20170819180509","C":"383a43293c363e683c423b1d1d1d1d5f2443a363189j56","CERT_ALL_DESCRIPT":"","DESCRIPTION":"1.建置與開發網站前後台\n2.資料庫建置與規劃\n3.程式開發與維護\n4.熟悉.net(C#、VB) C#尤佳，及MS SQL\n5.熟悉Ajax、Java Script、CSS等相關網頁開發技術\n6.熟MVC開發架構佳\n7.具備三層式架構，N-Tier及OOP經驗，若有Team Foundation 經驗佳","DIS_LEVEL":"0","DIS_LEVEL2":"0","DIS_LEVEL3":"0","DIS_ROLE":"0","DIS_ROLE2":"0","DIS_ROLE3":"0","DRIVER":"0","HANDICOMPENDIUM":"0","INDCAT":"資訊軟體服務","J":"5d49426c335b3e55303139622b2d361c93f39416b3f453e2020202064373a2524290j00","JOB":"Asp.Net ,C# 程式設計師","JOBCAT_DESCRIPT":"軟體設計工程師@Internet程式設計師@MIS程式設計師","JOBSKILL_ALL_DESC":"","JOB_ADDRESS":"林森南路4之2號7樓","JOB_ADDR_NO_DESCRIPT":"台北市中正區","LANGUAGE1":"001111","LANGUAGE2":"001111

In [4]:
// 對Array裡的每個Response做統計處理
// 找出各地區工作的總量

// 準備好一個空的Map資料結構
var forCalculate = traditionArray.clone
var jobCountMap= scala.collection.mutable.Map.empty[JsValue,Int]

// 透過迴圈的方式，逐筆把數據內的地區位置做統計
for (i <- 0 until forCalculate.length){
  for ( locate <- Json.parse(forCalculate(i)).\("data").\\("ADDR_NO_DESCRIPT")){
    jobCountMap.update(locate,jobCountMap.getOrElse(locate,0)+1)
  }
}

// println(jobCountMap)

forCalculate: ArrayBuffer[String] = ArrayBuffer(
  """
{"RECORDCOUNT":"4658","PAGECOUNT":"50","PAGE":"1","TOTALPAGE":"94","data":[{"ADDRESS":"林森南路4之2號7樓","ADDR_INDZONE":"","ADDR_NO_DESCRIPT":"台北市中正區","APPEAR_DATE":"20170819","APPEAR_TIME":"20170819180509","C":"383a43293c363e683c423b1d1d1d1d5f2443a363189j56","CERT_ALL_DESCRIPT":"","DESCRIPTION":"1.建置與開發網站前後台\n2.資料庫建置與規劃\n3.程式開發與維護\n4.熟悉.net(C#、VB) C#尤佳，及MS SQL\n5.熟悉Ajax、Java Script、CSS等相關網頁開發技術\n6.熟MVC開發架構佳\n7.具備三層式架構，N-Tier及OOP經驗，若有Team Foundation 經驗佳","DIS_LEVEL":"0","DIS_LEVEL2":"0","DIS_LEVEL3":"0","DIS_ROLE":"0","DIS_ROLE2":"0","DIS_ROLE3":"0","DRIVER":"0","HANDICOMPENDIUM":"0","INDCAT":"資訊軟體服務","J":"5d49426c335b3e55303139622b2d361c93f39416b3f453e2020202064373a2524290j00","JOB":"Asp.Net ,C# 程式設計師","JOBCAT_DESCRIPT":"軟體設計工程師@Internet程式設計師@MIS程式設計師","JOBSKILL_ALL_DESC":"","JOB_ADDRESS":"林森南路4之2號7樓","JOB_ADDR_NO_DESCRIPT":"台北市中正區","LANGUAGE1":"001111","LANGUAGE2":"001111","LANGUAGE3":"001111","LAT":"25.043836","LINK":"http://www.frog-

In [5]:
// 挑出前十個工作機會多的地區

// 此二句係以 declarative style，可以稍微觀察到這種寫法，在對於了解該階段時的資料長相時，是有所助益的
val sortJobCountMap = jobCountMap.toSeq.sortBy( record => record._2 ) 
val topTen = sortJobCountMap.takeRight(10)



sortJobCountMap: Seq[(JsValue, Int)] = ArrayBuffer(
  ("彰化縣和美鎮", 1),
  ("基隆市七堵區", 1),
  ("台中市梧棲區", 1),
  ("江蘇省蘇州市", 1),
  ("高雄市三民區", 1),
  ("彰化縣線西鄉", 1),
  ("彰化縣鹿港鎮", 1),
  ("台中市大里區", 1),
  ("美國加利佛尼亞州", 1),
  ("雲林縣虎尾鎮", 1),
  ("彰化縣大村鄉", 1),
  ("台南市仁德區", 1),
  ("廣東省東莞市", 1),
  ("台中市大甲區", 1),
  ("高雄市路竹區", 1),
  ("台中市霧峰區", 1),
  ("新加坡", 1),
  ("安徽省合肥市", 1),
  ("彰化縣彰化市", 1),
...
topTen: Seq[(JsValue, Int)] = ArrayBuffer(
  ("新北市板橋區", 64),
  ("新北市中和區", 86),
  ("台北市南港區", 91),
  ("新竹市", 113),
  ("台北市中山區", 140),
  ("台北市松山區", 143),
  ("台北市中正區", 150),
  ("台北市信義區", 165),
  ("台北市大安區", 270),
  ("台北市內湖區", 325)
)

In [6]:
// 整理成vegas所需要的可視化資料格式

var visualTopTen = ArrayBuffer.empty[Map[String,Any]];

for(i <- 0 until topTen.length ){
  visualTopTen.+=(
    Map("Country"->topTen(i)._1,"jobCount"-> topTen(i)._2)
  )
}

import vegas._
import vegas.data.External._

// 轉成可視化的網頁

val showHtml = Vegas("Java Job Count by each location")
  .withData( visualTopTen )
  .encodeX("Country", Ordinal)
  .encodeY("jobCount", Quantitative)
  .mark(Bar).html.pageHTML()

println(showHtml)

<html>
  <head>
    <script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/d3/3.5.17/d3.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega/2.6.3/vega.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega-lite/1.2.0/vega-lite.min.js" charset="utf-8"></script>
<script src="https://vega.github.io/vega-editor/vendor/vega-embed.js" charset="utf-8"></script>
  </head>
  <body>
 <script>
   var embedSpec = {
     mode: "vega-lite",
     spec: {
  "mark" : "bar",
  "encoding" : {
    "x" : {
      "field" : "Country",
      "type" : "ordinal"
    },
    "y" : {
      "field" : "jobCount",
      "type" : "quantitative"
    }
  },
  "description" : "Java Job Count by each location",
  "data" : {
    "values" : [
      {
        "Country" : "\"新北市板橋區\"",
        "jobCount" : 64
      },
      {
        "Country" : "\"新北市中和區\"",
        "jobCount" : 86
      },
      {
        "Country" : 

visualTopTen: ArrayBuffer[Map[String, Any]] = ArrayBuffer(
  Map("Country" -> "新北市板橋區", "jobCount" -> 64),
  Map("Country" -> "新北市中和區", "jobCount" -> 86),
  Map("Country" -> "台北市南港區", "jobCount" -> 91),
  Map("Country" -> "新竹市", "jobCount" -> 113),
  Map("Country" -> "台北市中山區", "jobCount" -> 140),
  Map("Country" -> "台北市松山區", "jobCount" -> 143),
  Map("Country" -> "台北市中正區", "jobCount" -> 150),
  Map("Country" -> "台北市信義區", "jobCount" -> 165),
  Map("Country" -> "台北市大安區", "jobCount" -> 270),
  Map("Country" -> "台北市內湖區", "jobCount" -> 325)
)
import vegas._
import vegas.data.External._
showHtml: String = """
<html>
  <head>
    <script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/d3/3.5.17/d3.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega/2.6.3/vega.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega-lite/1.2.0/vega-lite.min.js" charset="utf-8"></script>
<script src="https:/

In [7]:
// 將靜態網頁寫成檔案
var newHtmlFile = new java.io.FileWriter("104-Demo-Imperative.html",false)

try{
  newHtmlFile.write(showHtml)
}finally{
  newHtmlFile.close()
}

newHtmlFile: java.io.FileWriter = java.io.FileWriter@42c6c97c